**D3APL: Aplicações em Ciência de Dados** <br/>
IFSP Campinas

Prof. Dr. Samuel Martins (Samuka) <br/><br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# Animal Dataset - v2
We will evaluate some **multiclass classification** CNNs to predict the classes of the **Animal Dataset**: https://www.kaggle.com/datasets/alessiocorrado99/animals10


Target goals:
- Data Augmentation

## 1. Set up

#### 1.1 TensorFlow

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

**GPU available?**

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### 1.2 Fixing the seed for reproducibility (optional)
That's a try for reprodubility in Keras. See more on:
- https://stackoverflow.com/a/59076062
- https://machinelearningmastery.com/reproducible-results-neural-networks-keras/

In [ ]:
import os
import tensorflow as tf
import numpy as np
import random

def reset_random_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    
# make some random data
reset_random_seeds()

### 1.3. Dataset
**Animal Dataset**: https://www.kaggle.com/datasets/alessiocorrado99/animals10

#### 1.3.1 Load the preprocessed dataset
**Preprocessed dataset**: _'../datasets/animals-dataset/preprocessed/'_

In [ ]:
import numpy as np

In [ ]:
X_train = np.load('../datasets/animals-dataset/preprocessed/train_data_64x64x3.npy')
y_train = np.load('../datasets/animals-dataset/preprocessed/train_labels.npy')

X_val = np.load('../datasets/animals-dataset/preprocessed/validation_data_64x64x3.npy')
y_val = np.load('../datasets/animals-dataset/preprocessed/validation_labels.npy')

X_test = np.load('../datasets/animals-dataset/preprocessed/test_data_64x64x3.npy')
y_test = np.load('../datasets/animals-dataset/preprocessed/test_labels.npy')

In [ ]:
print(f'X_train.shape: {X_train.shape}')
print(f'y_train.shape: {y_train.shape}\n')

print(f'X_val.shape: {X_val.shape}')
print(f'y_val.shape: {y_val.shape}\n')

print(f'X_test.shape: {X_test.shape}')
print(f'y_test.shape: {y_test.shape}')

In [ ]:
# show a training image sample
import matplotlib.pyplot as plt

plt.imshow(X_train[0])

## 2. Building and Training a CNN via Keras

### 2.1 Data Augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

A **regularization method** that _perturbs_ training examples, _changing their appearance slightly_, before passing them into the network for training.

Data augmentation _artificially_ **increases** the size of the _training set_ by generating many _realistic variants_ of each _training instance_. <br/>
This _reduces overfitting_, making this a **regularization technique**. <br/>
The _generated instances_ should be as _realistic as possible_: ideally, given an image from the augmented training set, a human should not be able to tell whether it was augmented or not. 

The end result is that a _network_ consistently sees **“new” training data examples** generated from the _original training data_, partially alleviating the need for us to gather more training data (though in general, gathering more training data will rarely hurt your algorithm).

Common data augmentation techniques involve applying simple **geometric transformations** such as:
1. Translations
2. Rotations
3. Changes in scale
4. Shearing
5. Horizontal (and in some cases, vertical) flip

<img src='./figs/data_augmentation.png' width=800 />

https://nanonets.com/blog/data-augmentation-how-to-use-deep-learning-when-you-have-limited-data-part-2/

**IMPORTANT:** Applying a _(small) amount of these transformations_ to an input image will _change its **appearance** slightly_, but **IT DOES NOT CHANGE** the _class label_.

#### **A simple Example**

In [ ]:
# get an image sample from the database
img = X_train[9]

plt.imshow(img)

In [ ]:
# Reshape the input image to have an extra dimension ==> this is required by Keras data augmentation methods
# the first dimension will be the the total number of images which will be processed by Keras Data Augmentation
print(f'Shape before: {img.shape}')

img = np.expand_dims(img, axis=0)

print(f'Shape after: {img.shape}')

In [ ]:
# A data augmentation "model" with only changes in Rotation.
# We’ll allow our input images to be randomly rotated ± 30 degrees

from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation


# Randomly flips images horizontally during training.
# Randomly rotates images during TRAINING (not in the inference time).
# Factor indicates the interval in which the rotation angle can be. In this case: [-10% * 2pi, 10% * 2pi] = [-36°, +36°] 
# The input shape to this model is: (..., height, width, channels)
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(factor=0.1)
])

In [ ]:
# let's generate for new images by data augmentation

plt.figure(figsize=(10, 10))

for i in range(9):
    augmented_img = data_augmentation(img)
    
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_img[0])
    plt.axis("off")

#### **A more complex example**

In [ ]:
# get a batch of 2 images
X_train_batch = X_train[[9, 100]]

X_train_batch.shape

In [ ]:
plt.imshow(X_train_batch[0])

In [ ]:
plt.imshow(X_train_batch[1])

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(factor=0.1),
    RandomTranslation(height_factor=0.1, width_factor=0.1)
])

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(20, 20))

for i in range(5):
    augmented_X_batch = data_augmentation(X_train_batch)

    axs[0, i].imshow(augmented_X_batch[0])
    axs[1, i].imshow(augmented_X_batch[1])

### 2.2 Defining the Network Architecture
The simplest way to use the Keras preprocessing layers is making the _preprocessing layers_ **part of your model**: <br/>
https://www.tensorflow.org/tutorials/images/data_augmentation#option_1_make_the_preprocessing_layers_part_of_your_model


That's a simple CNN for _Multiclass Classification_:
- INPUT [64x64x3]
- **PREPROCESSING LAYERS**
- CONV [32, 4x4x3, 'valid']
- RELU
- MAX_POOL [2x2, stride=(1,1)]
- CONV [32, 4x4x3, 'valid']
- RELU => MAX_POOL [2x2, stride=(1,1)]
- FLATTEN
- FC [256]
- RELU => FC [10, 'softmax']  # number of classes

- optimizer: SGD with `learning_rate=0.01`
- kernel_initializer: "glorot_uniform"
- bias_initializer: "zeros"
- **Early stopping**

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation

def build_cnn(input_shape, n_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        
        # data augmentation layers
        RandomFlip("horizontal"),
        RandomRotation(factor=0.1),
        RandomTranslation(height_factor=0.1, width_factor=0.1),
        
        # CNN
        Conv2D(filters=32, kernel_size=(4,4), activation='relu'),
        MaxPool2D(pool_size=(2,2)),
        Conv2D(filters=32, kernel_size=(4,4), activation='relu'),
        MaxPool2D(pool_size=(2,2)),

        Flatten(),

        Dense(256, activation='relu'),
        Dense(n_classes, activation='softmax')
    ])
    
    return model

In [ ]:
input_shape = (64, 64, 3)
n_classes = np.unique(y_train).size

model = build_cnn(input_shape, n_classes)
opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
# vertical
plot_model(model, show_shapes=True, show_layer_activations=True)

##### **Observations**

- **Data augmentation** will run on-device, synchronously with the rest of your layers, and benefit from _GPU acceleration_.
- **Data augmentation** is **inactive at test time (inference time)** so input images will _only_ be augmented during calls to `Model.fit` (**not** `Model.evaluate` or `Model.predict`).

### 2.3 Training with Early Stopping

In case of GPU drivers, we can monitor its use by [_gpustat_](https://github.com/wookayin/gpustat).

On terminal, use: `gpustat -cpi`


In [ ]:
X_train.shape

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping_cb])

#### **Visualizing the training history**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

history_df = pd.DataFrame(history.history)

In [ ]:
history_df[['loss', 'val_loss']].plot(figsize=(8, 5))
plt.grid(True)
# plt.xticks(range(100))
plt.xlabel('Epochs')
plt.ylabel('Score')

history_df[['accuracy', 'val_accuracy']].plot(figsize=(8, 5))
plt.grid(True)
# plt.xticks(range(100))
plt.xlabel('Epochs')
plt.ylabel('Score')

## 3. Evaluating and Predicting New Samples by using our Overfitted Model

#### **Evaluation**
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#evaluate

In [ ]:
model.evaluate(X_test, y_test)

#### **Prediction**
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#predict

In [ ]:
y_test_proba = model.predict(X_test)
y_test_proba

#### **Class Prediction**
https://stackoverflow.com/a/69503180/7069696

In [ ]:
y_test_pred = np.argmax(y_test_proba, axis=1)
y_test_pred

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))

We got a better accuracy than the model without **data augmentation**.